# Running body condition functions outside of CollatriX

## Set up

### import packages

In [1]:
import pandas as pd
import numpy as np
import os, sys
import math
from scipy.integrate import quad

### set wd and import csv w/ measurements

In [3]:
#you're going to need to change the filepahts
###KC##
# wd = r'/Users/kcbierlich/Dropbox/GitHub/Whale_Body_Condition_Analysis/'
###CLARA#####
wd = "/Users/clarabird/github/Whale_Body_Condition_Analysis"

csv = os.path.join(wd,"Data/R_outputs/whales_merged.csv")

#read in csv
df = pd.read_csv(csv)
display(df)

,Unnamed: 0,Animal_ID,Reproductive_Class,Image,TL,TL-10.00% Width,TL-15.00% Width,TL-20.00% Width,TL-25.00% Width,TL-30.00% Width,...,reproductive._Class.original,Lessthan20.,Exactly20.,Between20and25.,Exactly25.,Greaterthan25.,eye_width,head,tailstock,Range
0,1,Bb180304_L_F1-01-42,MinkeUnknown,180304_L_F1_DSC09027.JPG,8.100000,0.790000,0.990000,1.200000,1.240000,1.320000,...,NaN,0,1,0,0,0,Exactly20%,20%,85%,20%-85%
1,2,Bb180304_L_F1-03,MinkeUnknown,180304_L_F1_DSC09096.JPG,8.970000,0.900000,1.050000,1.350000,1.440000,1.500000,...,NaN,0,1,0,0,0,Exactly20%,20%,85%,20%-85%
2,3,Bb180304_L_F1-04-45,MinkeUnknown,180304_L_F1_DSC09069.JPG,8.700000,0.840000,1.020000,1.230000,1.290000,1.330000,...,NaN,1,0,0,0,0,Lessthan20%,20%,85%,20%-85%
3,4,Bb180304_L_F4-11,MinkeUnknown,180304_L_F4_DSC09331.JPG,8.680000,0.900000,1.100000,1.390000,1.420000,1.460000,...,NaN,1,0,0,0,0,Lessthan20%,20%,85%,20%-85%
4,5,Bb180307_L_F7-01,MinkeUnknown,180307_L_F7_DSC00437.JPG,6.690000,0.670000,0.830000,0.990000,1.060000,1.180000,...,NaN,0,1,0,0,0,Exactly20%,20%,85%,20%-85%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,231,Triplet,Lactating female,160829_F2_triplet.png,12.435340,0.825472,1.278151,1.677573,1.837342,2.050367,...,Mom,0,1,0,0,0,Exactly20%,20%,85%,20%-85%
231,232,Tulip,Resting female,170706_F1_Tulip.png,14.665633,0.887958,1.288972,1.604054,1.833204,1.833204,...,RestingFemale,0,1,0,0,0,Exactly20%,20%,85%,20%-85%
232,233,White_Hole,Resting female,171005_F1_WhiteHole.png,14.672634,1.071597,1.401319,1.840949,2.005810,2.390485,...,RestingFemale,0,1,0,0,0,Exactly20%,20%,85%,20%-85%
233,234,White_Knuckle,Calf,160829_F2_whiteknuckle.png,7.325178,0.614887,0.802027,0.935698,1.015901,1.176306,...,Calf,1,0,0,0,0,Lessthan20%,20%,85%,20%-85%


### set up subset dataframes by the individuals w/ different ranges

In [8]:
dfw = df.loc[df['Range'] == '20%-85%']
dfx = df.loc[df['Range'] == '25-85']
dfy = df.loc[df['Range'] == '20-90']
dfz = df.loc[df['Range'] == '20-90']

## Define Functions and run

### BODY VOLUME

#### function

In [ ]:
def body_vol(df_all,tl_name,interval,lower,upper):
    body_name = "BV" #name of body volume column will use interval amount
    volm = [] #make empty list of widths
    for x in range(lower,(upper + interval), interval): #loop through range of widths
        xx = "{0}-{1}.0%".format(tl_name,str(x)) #create the name of the headers to pull measurements from
        volm += [xx] #add to list
    vlist = []
    for i in volm: #loop through column headers
        for ii in df_all.columns:
            if i in ii:
                vlist += [ii]
    ids = []; vs = []; imgs = []
    for i,j in enumerate(vlist[:-1]):
        jj = vlist[i+1]
        #calculate volume by looping through two columns at a time
        for rr, RR, hh,anid,img in zip(df_all[j],df_all[jj], df_all[tl_name],df_all['Animal_ID'],df_all['Image']):
            ph = float(interval)/float(100); h = float(hh)*ph
            r = float(rr)/float(2); R = float(RR)/float(2)
            v1 = (float(1)/float(3))*(math.pi)*h*((r**2)+(r*R)+(R**2))
            ids += [anid]; vs += [v1]; imgs += [img]
    d = {'Animal_ID':ids, body_name:vs, 'Image':imgs} #make dataframe of id and body volume
    df = pd.DataFrame(data = d) #make dataframe
    cls = df.columns.tolist() #get list of column headers
    grBy = ['Animal_ID','Image'] #list of columns to group by
    groups = [x for x in cls if x not in grBy] #get list of columns to be grouped
    df1 = df.groupby(['Animal_ID','Image'])[groups].apply(lambda x: x.astype(float).sum()).reset_index() #group to make sure no duplicates
    df2 = df1.merge(df_all[['Animal_ID','Species','Reproductive_Class','TL']],on=['Animal_ID'])
    return df2

#### run in loop

In [ ]:
#make empty dataframe to fill with body volume
df_bv = pd.DataFrame(data={})
#run function for each of the four dataframes
wv = body_vol(dfw,"TL",5,20,85)
xv = body_vol(dfx,"TL",5,25,85)
yv = body_vol(dfy,"TL",5,20,90)
zv = body_vol(dfz,"TL",5,25,90)
#concatenate back into one big dataframe
df_bv = pd.concat([wv,xv,yv,zv])

### BAI
output will include surface area and BAI

#### function

In [11]:
def bai_trapezoid(df_all,tl_name,b_interval,b_lower,b_upper):
    bai_name = "BAItrap_{0}%".format(b_interval) #create BAI column header using interval
    bai = [] #list of columns containing the width data we want to use to calculate BAI
    for x in range(b_lower,(b_upper + b_interval), b_interval): # loop through columns w/in range we want
        xx = "{0}-{1}.00%".format(tl_name,str(x)) #set up column name
        bai += [xx]
    blist = []
    for i in bai:
        for ii in df_all.columns:
            if i in ii:
                blist += [ii]
    ids = []
    sas = []
    imgs = []
    for i,j in enumerate(blist[:-1]):
        jj = blist[i+1]
        for w, W, hh,anid,img in zip(df_all[j],df_all[jj], df_all[tl_name],df_all['Animal_ID'],df_all['Image']):
            ph = float(b_interval)/float(100)
            h = float(hh)*ph
            sa1 = (float(1)/float(2))*(w+W)*h
            ids += [anid]
            sas += [sa1]
            imgs += [img]
    d = {'Animal_ID':ids, 'SA':sas, 'Image':imgs}
    df = pd.DataFrame(data = d)

    cls = df.columns.tolist()
    grBy = ['Animal_ID','Image']
    groups = [x for x in cls if x not in grBy]
    df1 = df.groupby(['Animal_ID','Image'])[groups].apply(lambda x: x.astype(float).sum()).reset_index()
    dft = pd.merge(df_all[['Animal_ID','Image',tl_name]],df1,on = ['Animal_ID','Image'])
    dft[bai_name] = (dft['SA']/((dft[tl_name]*((b_upper-b_lower)/float(100)))**2))*100
    dft = dft.drop([tl_name],axis=1)
    return dft

In [ ]:
df_all = dfw
tl_name = "TL"
b_interval = 5
b_lower = 20
b_upper = 60
vname = "help"

In [28]:
def bai_parabola(df_all,tl_name,b_interval,b_lower,b_upper,vname):
    df_all = df_all.dropna(how="all",axis='rows').reset_index()

    bai_name = "BAIpar_{0}%".format(vname) #create BAI column header using interval
    bai = [] #list of columns containing the width data we want to use to calculate BAI
    perc_l = []
    for x in range(b_lower,(b_upper + b_interval), b_interval): # loop through columns w/in range we want
        xx = "{0}-{1}.00%".format(tl_name,str(x)) #set up column name
        bai += [xx]
        perc_l += [x/100]
    #here we check that the widths are actually in the column headers
    blist = []
    for i in bai:
        for ii in df_all.columns:
            if i in ii:
                blist += [ii]

    #make empty lists to be filled
    ids = []
    bais = []
    imgs = []
    #loop through the dataframe by image/ID
    for img,anid in zip(df_all['Image'],df_all['Animal_ID']):
        idx = df_all.loc[df_all['Image'] == img].index[0]
        ids += [anid]
        imgs += [img]
        #fill list of y values (y = width)
        ylist = []
        for b in blist:
            ylist += [(df_all[b].tolist()[idx])] #populate y values withwidth at each incr.
        ylist = np.array(ylist)

        tl = df_all[tl_name].tolist()[idx]
        min_tl = tl*(b_lower/100)
        max_tl = tl*(b_upper/100)

        xlist = [x*tl for x in perc_l] #populate x vlaues with x of TL at each incr.
        xlist = np.array(xlist)

        #make list of 500 x values along TL between bounds
        newx = np.linspace(min_tl,max_tl,500)

        #fit quadratric linear model using original x and y lists. then fit to big list of x values
        lm = np.polyfit(xlist,ylist,2)
        fit = np.poly1d(lm)
        pred = fit(newx)

        #integrate using linear model to get surface area
        I = quad(fit,min_tl,max_tl)
        sa = I[0]

        #calculate BAI
        bai = (sa/((tl*((b_upper-b_lower)/float(100)))**2))*100

        bais += [bai]

    d = {'Animal_ID':ids, bai_name:bais, 'Image':imgs}
    df = pd.DataFrame(data = d)

    cls = df.columns.tolist()
    grBy = ['Animal_ID','Image']
    groups = [x for x in cls if x not in grBy]
    dfp = df.groupby(['Animal_ID','Image'])[groups].apply(lambda x: x.astype(float).sum()).reset_index()
    return dfp


#### run in loop

In [30]:
#make empty dataframe to fill with body volume
df_bai = pd.DataFrame(data={})
#run function for each of the four dataframes
wb = bai_parabola(dfw,"TL",5,20,85,"fuck")
xb = bai_trapezoid(dfx,"TL",5,25,85)
yb = bai_trapezoid(dfy,"TL",5,20,90)
zb = bai_trapezoid(dfz,"TL",5,25,90)
#concatenate back into one big dataframe
df_bai = pd.concat([wb,xb,yb,zb])

display(wb)

,Animal_ID,Image,BAIpar_fuck%
0,Bb180304_L_F1-01-42,180304_L_F1_DSC09027.JPG,20.135215
1,Bb180304_L_F1-03,180304_L_F1_DSC09096.JPG,20.116148
2,Bb180304_L_F1-04-45,180304_L_F1_DSC09069.JPG,19.063934
3,Bb180304_L_F4-11,180304_L_F4_DSC09331.JPG,20.339587
4,Bb180307_L_F7-01,180307_L_F7_DSC00437.JPG,21.540679
...,...,...,...
130,Triplet,160829_F2_triplet.png,17.768901
131,Tulip,170706_F1_Tulip.png,15.197530
132,White_Hole,171005_F1_WhiteHole.png,17.482868
133,White_Knuckle,160829_F2_whiteknuckle.png,20.765655


## Merge back together

In [ ]:
df_done = df_bv.merge(df_bai,on=['Animal_ID','Image'])
df_done
df_done.to_csv("bodycond_variablerange.csv")